In [2]:
import pandas as pd
from glob import glob
from tqdm import tqdm
import re
from sklearn.model_selection import train_test_split
import PIL
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import MaxPooling3D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import ConvLSTM2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv3D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras.layers import GlobalAveragePooling2D

In [2]:
import tensorflow.keras.applications as cnn

# train데이터 불러오기 -> 둘 중에 하나 선택(CNN or CNN+LSTM)

In [39]:
time_steps = 100

In [40]:
# train 파일 불러오기(CNN+LSTM) 
classes = np.identity(n=6, dtype='int')
p = re.compile('[0-9]+')
label_df = pd.read_csv('labels.csv')
images = []
targets = []
stratify = []
for n in range(142):
    folder = 'D:/train/file_{}'.format(n)
    file_num_list = sorted(list(map(lambda x : int(p.findall(x)[-1]),glob(folder+'/*'))))
    length = len(file_num_list)
    select_num_list = [file_num_list[length*int(idx/time_steps)-1] for idx in range(1,time_steps+1)]
    image_list = list(map(lambda x : folder + '\\' +str(x) + '.jpg',select_num_list))
    label = label_df.loc[label_df['file_name']=='file_'+str(n)]['label'].values[0]
    for image_path in image_list:
        img = image.load_img(image_path,target_size=(64,64))
        img = image.img_to_array(img)
        img = img/255
        images.append(img)
    targets.append(classes[label])
    stratify.append(label)
        
# images shape 변경
images = np.array(images)
targets = np.array(targets)
images = images.reshape(images.shape[0]//time_steps,time_steps,images.shape[1],images.shape[2],images.shape[3])

In [18]:
images.shape

(1420, 64, 64, 3)

In [ ]:
# # train 파일 불러오기(CNN)
# classes = np.identity(n=6, dtype='int')
# p = re.compile('[0-9]+')
# label_df = pd.read_csv('labels.csv')
# images = []
# targets = []
# stratify = []
# for n in range(142):
#     folder = 'D:/train/file_{}'.format(n)
#     file_num_list = sorted(list(map(lambda x : int(p.findall(x)[-1]),glob(folder+'/*'))))
#     length = len(file_num_list)
#     select_num_list = [file_num_list[length*int(idx/time_steps)-1] for idx in range(1,time_steps+1)]
#     image_list = list(map(lambda x : folder + '\\' +str(x) + '.jpg',select_num_list))
#     label = label_df.loc[label_df['file_name']=='file_'+str(n)]['label'].values[0]
#     for image_path in image_list:
#         img = image.load_img(image_path,target_size=(64,64))
#         img = image.img_to_array(img)
#         img = img/255
#         images.append(img)
#         targets.append(classes[label])
#         stratify.append(label)
        
# # images shape 변경
# images = np.array(images)
# targets = np.array(targets)
# images = images.reshape(images.shape[0]//time_steps,time_steps,images.shape[1],images.shape[2],images.shape[3])

# train_set, test_set 분리

In [41]:
# train set, tset set 분리
train_idx, valid_idx = train_test_split(range(images.shape[0]), test_size = 0.2, stratify = stratify)

X_train = images[train_idx]
y_train = targets[train_idx]

X_valid = images[valid_idx]
y_valid = targets[valid_idx]

print('X_train shape : ', X_train.shape)
print('X_valid shape : ', X_valid.shape)
print('y_train shape : ', y_train.shape)
print('y_valid shape : ', y_valid.shape)

X_train shape :  (113, 100, 64, 64, 3)
X_valid shape :  (29, 100, 64, 64, 3)
y_train shape :  (113, 6)
y_valid shape :  (29, 6)


# CNN 모델

In [191]:
model = Sequential()
model.add(Conv2D(input_shape=(224//4,224//4,3),filters=64//4,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64//4,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128//4, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128//4, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256//4, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256//4, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256//4, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512//4, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512//4, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512//4, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512//4, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512//4, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512//4, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(GlobalAveragePooling2D())


model.add(Dense(units=4096//4,activation="relu"))
model.add(Dense(units=4096//4,activation="relu"))
model.add(Dense(units=6, activation="softmax"))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
# 모델 요약
model.summary()

# CNN+LSTM 모델

In [8]:
model = Sequential()
model.add(TimeDistributed(Conv2D(input_shape=(None,100,224//4,224//4,3),filters=64//4,kernel_size=(3,3),padding="same", activation="relu")))
model.add(TimeDistributed(Conv2D(filters=64//4,kernel_size=(3,3),padding="same", activation="relu")))
model.add(TimeDistributed(MaxPool2D(pool_size=(2,2),strides=(2,2))))
model.add(TimeDistributed(Conv2D(filters=128//4, kernel_size=(3,3), padding="same", activation="relu")))
model.add(TimeDistributed(Conv2D(filters=128//4, kernel_size=(3,3), padding="same", activation="relu")))
model.add(TimeDistributed(MaxPool2D(pool_size=(2,2),strides=(2,2))))
model.add(TimeDistributed(Conv2D(filters=256//4, kernel_size=(3,3), padding="same", activation="relu")))
model.add(TimeDistributed(Conv2D(filters=256//4, kernel_size=(3,3), padding="same", activation="relu")))
model.add(TimeDistributed(Conv2D(filters=256//4, kernel_size=(3,3), padding="same", activation="relu")))
model.add(TimeDistributed(MaxPool2D(pool_size=(2,2),strides=(2,2))))
model.add(TimeDistributed(Conv2D(filters=512//4, kernel_size=(3,3), padding="same", activation="relu")))
model.add(TimeDistributed(Conv2D(filters=512//4, kernel_size=(3,3), padding="same", activation="relu")))
model.add(TimeDistributed(Conv2D(filters=512//4, kernel_size=(3,3), padding="same", activation="relu")))
model.add(TimeDistributed(MaxPool2D(pool_size=(2,2),strides=(2,2))))
model.add(TimeDistributed(Conv2D(filters=512//4, kernel_size=(3,3), padding="same", activation="relu")))
model.add(TimeDistributed(Conv2D(filters=512//4, kernel_size=(3,3), padding="same", activation="relu")))
model.add(TimeDistributed(Conv2D(filters=512//4, kernel_size=(3,3), padding="same", activation="relu")))
model.add(TimeDistributed(MaxPool2D(pool_size=(2,2),strides=(2,2))))
model.add(TimeDistributed(GlobalAveragePooling2D()))

model.add(LSTM(1024//4, activation='relu', return_sequences=False))

model.add(Dense(units=4096//4,activation="relu"))
model.add(Dense(units=4096//4,activation="relu"))
model.add(Dropout(.5))
model.add(Dense(units=6, activation="softmax"))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [9]:
# 모델 요약
model.build((None,100,64,64,3))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 100, 64, 64, 16)   448       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 100, 64, 64, 16)   2320      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 100, 32, 32, 16)   0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 100, 32, 32, 32)   4640      
_________________________________________________________________
time_distributed_4 (TimeDist (None, 100, 32, 32, 32)   9248      
_________________________________________________________________
time_distributed_5 (TimeDist (None, 100, 16, 16, 32)   0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 100, 16, 16, 64)   1

In [197]:
# 모델 학습(공통)
model.fit(X_train,y_train,
                         steps_per_epoch = 100,
                         epochs =1,
                         validation_data = (X_valid,y_valid),
                         validation_steps =1)

100/100 [==============================] - 59s 545ms/step - loss: 1.8004 - accuracy: 0.1150 - val_loss: 1.7917 - val_accuracy: 0.1724


In [10]:
# test 파일 불러오기(CNN+LSTM)
test_set = []
for n in range(142,187):
    folder = 'D:/new_images/test/test/file_{}'.format(n)
    file_num_list = sorted(list(map(lambda x : int(p.findall(x)[-1]),glob(folder+'/*'))))
    length = len(file_num_list)
    select_num_list = [file_num_list[length*int(idx/time_steps)-1] for idx in range(1,time_steps+1)]
    image_list = list(map(lambda x : folder + '\\' +str(x) + '.jpg',select_num_list))
    for image_path in image_list:
        img = image.load_img(image_path,target_size=(64,64))
        img = image.img_to_array(img)
        img = img/255
        test_set.append(img)
   
        
# images shape 변경
test_set = np.array(test_set)
test_set = test_set.reshape(test_set.shape[0]//time_steps,time_steps,test_set.shape[1],test_set.shape[2],test_set.shape[3])

In [36]:
glob(folder+'/*')

['D:/new_images/test/test/file_186\\0.jpg',
 'D:/new_images/test/test/file_186\\1.jpg',
 'D:/new_images/test/test/file_186\\10.jpg',
 'D:/new_images/test/test/file_186\\100.jpg',
 'D:/new_images/test/test/file_186\\101.jpg',
 'D:/new_images/test/test/file_186\\102.jpg',
 'D:/new_images/test/test/file_186\\103.jpg',
 'D:/new_images/test/test/file_186\\104.jpg',
 'D:/new_images/test/test/file_186\\105.jpg',
 'D:/new_images/test/test/file_186\\106.jpg',
 'D:/new_images/test/test/file_186\\107.jpg',
 'D:/new_images/test/test/file_186\\108.jpg',
 'D:/new_images/test/test/file_186\\109.jpg',
 'D:/new_images/test/test/file_186\\11.jpg',
 'D:/new_images/test/test/file_186\\110.jpg',
 'D:/new_images/test/test/file_186\\111.jpg',
 'D:/new_images/test/test/file_186\\112.jpg',
 'D:/new_images/test/test/file_186\\113.jpg',
 'D:/new_images/test/test/file_186\\114.jpg',
 'D:/new_images/test/test/file_186\\115.jpg',
 'D:/new_images/test/test/file_186\\116.jpg',
 'D:/new_images/test/test/file_186\\117.

In [38]:
# test 파일 불러오기(CNN)
test_set = []
for n in range(142,187):
    folder = 'D:/new_images/test/test/file_{}'.format(n)
    image_list = glob(folder+'/*')
    for image_path in image_list:
        img = image.load_img(image_path,target_size=(64,64))
        img = image.img_to_array(img)
        img = img/255
        test_set.append(img)
   
        

test_set = np.array(test_set)
# test_set = images.reshape(test_set.shape[0]//time_steps,time_steps,test_set.shape[1],test_set.shape[2],test_set.shape[3])

In [39]:
test_set

array([[[[0.20784314, 0.25490198, 0.34117648],
         [0.20784314, 0.25490198, 0.34117648],
         [0.20784314, 0.25490198, 0.34117648],
         ...,
         [0.2       , 0.26666668, 0.3372549 ],
         [0.20392157, 0.27058825, 0.34117648],
         [0.20392157, 0.27058825, 0.34117648]],

        [[0.2       , 0.2509804 , 0.3254902 ],
         [0.2       , 0.2509804 , 0.3254902 ],
         [0.2       , 0.2509804 , 0.3254902 ],
         ...,
         [0.2       , 0.26666668, 0.3372549 ],
         [0.19607843, 0.2627451 , 0.33333334],
         [0.19607843, 0.2627451 , 0.33333334]],

        [[0.21960784, 0.2509804 , 0.3254902 ],
         [0.21176471, 0.24313726, 0.31764707],
         [0.21176471, 0.24313726, 0.31764707],
         ...,
         [0.2       , 0.26666668, 0.3372549 ],
         [0.2       , 0.26666668, 0.3372549 ],
         [0.2       , 0.26666668, 0.3372549 ]],

        ...,

        [[0.53333336, 0.49411765, 0.49019608],
         [0.5176471 , 0.47843137, 0.4745098 ]

In [34]:
output = model.predict(test_set)

print(output)

[[6.1216741e-03 6.7210807e-05 5.2229069e-02 9.3825799e-01 3.3090315e-03
  1.5024630e-05]
 [6.1216741e-03 6.7210807e-05 5.2229069e-02 9.3825799e-01 3.3090315e-03
  1.5024630e-05]
 [6.1216741e-03 6.7210807e-05 5.2229069e-02 9.3825799e-01 3.3090315e-03
  1.5024630e-05]
 ...
 [3.1429672e-04 1.8640743e-04 2.2888176e-04 7.5715902e-04 2.0730779e-04
  9.9830592e-01]
 [3.1429672e-04 1.8640743e-04 2.2888176e-04 7.5715902e-04 2.0730779e-04
  9.9830592e-01]
 [3.1429672e-04 1.8640743e-04 2.2888176e-04 7.5715902e-04 2.0730779e-04
  9.9830592e-01]]
